In [1]:
import matplotlib
import matplotlib.pyplot as plt
import os
import logging
import glob

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [2]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [3]:
%ls '/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/TENSOR_RESULTS'

efficientdet-d0/


In [4]:
model = "efficientdet-d0"
config = "bestmodel"
output_directory = f'/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/FINE_TUNED_MODEL/{model}/{config}'

In [5]:
import pathlib

filenames = list(pathlib.Path(os.path.join(output_directory,"checkpoint")).glob('*.index'))

filenames.sort()
print(filenames)

#recover our saved model
pipeline_file = f"/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/models/research/deploy/{model}/config_6/pipeline_file.config"
pipeline_config = pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = str(filenames[-1]).replace('.index','')
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(str(filenames[-1]).replace('.index','')))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    
    
    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

[PosixPath('/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/FINE_TUNED_MODEL/efficientdet-d0/bestmodel/checkpoint/ckpt-0.index')]


In [6]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [7]:
%cd /home/labuser/LogoDet/LogoDetection_DSBAProject/inference

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference


In [8]:
if "INFERENCE_DIR" not in os.listdir(os.getcwd()):
    os.mkdir(output_directory)
    logging.info("Creating the directory INFERENCE_DIRbecause it did not exist") 
else:
    logging.info("The directory INFERENCE_DIR is already present, files will be stored there") 

In [9]:
%cd /home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR


In [12]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random
from build_csv import build_predicion_CSV

TEST_IMAGE_PATHS = glob.glob('/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/*.jpg')

counter = 0
for image in TEST_IMAGE_PATHS:
    image_path = image
    print(image)
    #image_path = "/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/INFERENCE_DIR/NFLogo1.jpg"
    image_np = load_image_into_numpy_array(image_path)

    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    threshold = 0.80
    """
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True, # Needs to be true cause otherwise boxes go out of range
          max_boxes_to_draw=3, # Minimized to 10 because otherwise we might have troubles 
          min_score_thresh=threshold, # Minimized the score, as for now it seems to be too confident over Adidas
          agnostic_mode=False, # Set to False because it must be giving informations related to the boxes
    )"""
    
    build_predicion_CSV(threshold=threshold, name_image=image,counter = counter, detections= detections,category=category_index)
    counter = counter + 1

    """plt.figure(figsize=(12,16))
    plt.imshow(image_np_with_detections)
    plt.show()"""

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_1357350732949111450_20161009_jpg.rf.88234f398f606568c296e9dd1204072c.jpg
{'la_1357350732949111450_20161009_jpg.rf.88234f398f606568c296e9dd1204072c.jpg': {0: {'label': 5, 'accuracy': 0.7700003, 'prediction': [0.6790913, 0.6867538, 0.80621976, 0.8609331]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/london_1836721383621492519_20180802_jpg.rf.ffaf084b79138435fa466b830bbc9751.jpg
{'london_1836721383621492519_20180802_jpg.rf.ffaf084b79138435fa466b830bbc9751.jpg': {0: {'label': 5, 'accuracy': 0.83465266, 'prediction': [0.8692336, 0.8816118, 0.96484387, 0.9808042]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_2017_1590040429369621714_20170826_jpg.rf.7f5ec78f52535d4f7f18888a4a90353c.jpg
{'la_2017_1590040429369621714_20170826_jpg.rf.7f5ec78f52535d4f7f18888a4a90353c.jpg': {0: {'label': 1, 'accuracy': 0.83740103, 'prediction': [0.42822155, 0.33864108, 0.57128686,

{'phoenix_1916413683077196947_20181120_jpg.rf.85a54889fc768dcc1209e55a24540bd3.jpg': {0: {'label': 6, 'accuracy': 0.93123657, 'prediction': [0.554495, 0.28371882, 0.6318097, 0.387101]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/houston_1648619838425282237_20171115_jpg.rf.7657a9981f6082dec10a2fe4f8c64d09.jpg
{'houston_1648619838425282237_20171115_jpg.rf.7657a9981f6082dec10a2fe4f8c64d09.jpg': {0: {'label': 1, 'accuracy': 0.9417794, 'prediction': [0.5634289, 0.34361216, 0.7618166, 0.5770875]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_1282145010627793237_20160627_jpg.rf.46fa745d5e78f006f0b73a5a6dad26ba.jpg
{'la_1282145010627793237_20160627_jpg.rf.46fa745d5e78f006f0b73a5a6dad26ba.jpg': {0: {'label': 5, 'accuracy': 0.7224762, 'prediction': [0.6502817, 0.21589288, 0.7921633, 0.5168518]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/chicago_1187788708068303354_20160218_jpg.rf.03144839d65ef73bfd5033091

{'miami_1526234338344348709_20170530_jpg.rf.2b5f3e4d4d2d547c9a80bc838178b1d3.jpg': {0: {'label': 3, 'accuracy': 0.9756486, 'prediction': [0.85141814, 0.36251926, 1.0, 0.52436626]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/london_1943472452215849150_20181227_jpg.rf.a47ad8361fa37a8f48a8c420503eb337.jpg
{'london_1943472452215849150_20181227_jpg.rf.a47ad8361fa37a8f48a8c420503eb337.jpg': {0: {'label': 7, 'accuracy': 0.8944922, 'prediction': [0.5281762, 0.21787827, 0.72045755, 0.46165258]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/miami_1546601019859830734_20170627_jpg.rf.8d51c67ecb8a934e4eb1912e0d4f0523.jpg
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/charlotte_1609227143246490097_20170922_jpg.rf.22adbdfce92bb0ac96e1d962bef956a6.jpg
{'charlotte_1609227143246490097_20170922_jpg.rf.22adbdfce92bb0ac96e1d962bef956a6.jpg': {0: {'label': 5, 'accuracy': 0.75568634, 'prediction': [0.6902949, 0.24990304, 0.748

{'miami_1626296322113679515_20171015_jpg.rf.aeeb151304ce6dcf295ffd52cfc729ed.jpg': {0: {'label': 1, 'accuracy': 0.73414224, 'prediction': [0.42922178, 0.33155343, 0.70066035, 0.53414714]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/london_1842172358208893122_20180809_jpg.rf.4dbbbff0202c8e8450c39e1e8c3356d9.jpg
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/phoenix_1916486612990299842_20181120_jpg.rf.8fa9011cf9d0234a45c69eca4a4f7b00.jpg
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/chicago_1381628490462030446_20161112_jpg.rf.a8354508b82f49d303dfa6dff55acb38.jpg
{'chicago_1381628490462030446_20161112_jpg.rf.a8354508b82f49d303dfa6dff55acb38.jpg': {0: {'label': 7, 'accuracy': 0.96016455, 'prediction': [0.43016326, 0.45458996, 0.650439, 0.7087921]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_1219547226264362761_20160402_jpg.rf.37039e82ed01e5870e5afbbaa5f6931e.jpg
/home/labuser/L

{'miami_1687700511278862581_20180108_jpg.rf.4f3a09ea3640248ea0f305b935cf6472.jpg': {0: {'label': 2, 'accuracy': 0.95414096, 'prediction': [0.35311902, 0.6762635, 0.50154495, 0.7854782]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/houston_1803420408779800777_20180617_jpg.rf.2bb72f6e7ea7efeecc69226c41e2855f.jpg
{'houston_1803420408779800777_20180617_jpg.rf.2bb72f6e7ea7efeecc69226c41e2855f.jpg': {0: {'label': 9, 'accuracy': 0.8334922, 'prediction': [0.7729854, 0.5319826, 0.85325474, 0.6427862]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/chicago_1527256074616701241_20170601_jpg.rf.775f104fdac7680867470f6f4847d859.jpg
{'chicago_1527256074616701241_20170601_jpg.rf.775f104fdac7680867470f6f4847d859.jpg': {0: {'label': 1, 'accuracy': 0.8295337, 'prediction': [0.003818322, 0.55974287, 0.077508286, 0.655453]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/chicago_1497883604910757474_20170421_jpg.rf.cb12b6871ac

...

{'chicago_1443238891511818927_20170205_jpg.rf.9af7cae55a1155b24d796482c86eff90.jpg': {0: {'label': 4, 'accuracy': 0.92614466, 'prediction': [0.14802217, 0.37099588, 0.24435505, 0.45278174]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/london_1939507407915281004_20181221_jpg.rf.34676cd80d2afb18c99dc52b89273633.jpg
{'london_1939507407915281004_20181221_jpg.rf.34676cd80d2afb18c99dc52b89273633.jpg': {0: {'label': 3, 'accuracy': 0.9572911, 'prediction': [0.055727914, 0.4824057, 0.13734359, 0.56621814]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_2017_1585815658575121551_20170820_jpg.rf.51439f1ad5831eb61c5e0652c7a17795.jpg
{'la_2017_1585815658575121551_20170820_jpg.rf.51439f1ad5831eb61c5e0652c7a17795.jpg': {0: {'label': 2, 'accuracy': 0.7493053, 'prediction': [0.39349896, 0.91921306, 0.4695304, 0.99793863]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/miami_1942747667239183366_20181226_jpg.rf.6c9cca69e

{'miami_1553204546002789986_20170706_jpg.rf.ddb82c5a00879d42be267f68240e4914.jpg': {0: {'label': 1, 'accuracy': 0.87311447, 'prediction': [0.6115978, 0.27603906, 0.7080114, 0.39887595]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_1362958207085757058_20161017_jpg.rf.92395c73cb454928920504c37a849be7.jpg
{'la_1362958207085757058_20161017_jpg.rf.92395c73cb454928920504c37a849be7.jpg': {0: {'label': 5, 'accuracy': 0.90773666, 'prediction': [0.36245215, 0.21061265, 0.48245162, 0.42297906]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/cleveland_1295450869357369375_20160716_jpg.rf.801f93c719bc174de781c585f511334a.jpg
{'cleveland_1295450869357369375_20160716_jpg.rf.801f93c719bc174de781c585f511334a.jpg': {0: {'label': 5, 'accuracy': 0.74506783, 'prediction': [0.1136645, 0.5548518, 0.1740281, 0.7032486]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/houston_1901891080203619145_20181030_jpg.rf.4410d3e5de6cf5b6

{'miami_1290378706455389091_20160709_jpg.rf.84abf8a2a3888d09e81764d0f3532310.jpg': {0: {'label': 1, 'accuracy': 0.67358893, 'prediction': [0.89317125, 0.51941794, 0.9851623, 0.5862619]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/miami_1874124756636421733_20180922_jpg.rf.9f469b1d45769427d2f54d9acc28d824.jpg
{'miami_1874124756636421733_20180922_jpg.rf.9f469b1d45769427d2f54d9acc28d824.jpg': {0: {'label': 5, 'accuracy': 0.8696467, 'prediction': [0.022914894, 0.4508379, 0.090321355, 0.5967205]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/charlotte_1827890282436443331_20180720_jpg.rf.48a6df7d48777904e4c847b5fbc8d9d8.jpg
{'charlotte_1827890282436443331_20180720_jpg.rf.48a6df7d48777904e4c847b5fbc8d9d8.jpg': {0: {'label': 7, 'accuracy': 0.92804706, 'prediction': [0.7440019, 0.1880872, 0.8646912, 0.28864458]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/houston_1602232569457270421_20170912_jpg.rf.3a2c851c50

{'houston_1722871039052347285_20180225_jpg.rf.6055e006175b43218823480b2a32933f.jpg': {0: {'label': 8, 'accuracy': 0.81824386, 'prediction': [0.69454044, 0.80222315, 0.78321046, 0.903573]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/chicago_1356242152782302095_20161008_jpg.rf.d6c07585a7911c5929852fe1e2507f30.jpg
{'chicago_1356242152782302095_20161008_jpg.rf.d6c07585a7911c5929852fe1e2507f30.jpg': {0: {'label': 5, 'accuracy': 0.87075835, 'prediction': [0.6362744, 0.41333246, 0.75911814, 0.6409242]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/miami_1419204649332643233_20170103_jpg.rf.33b2f58ab0e0f81447051dc0e28c04b7.jpg
{'miami_1419204649332643233_20170103_jpg.rf.33b2f58ab0e0f81447051dc0e28c04b7.jpg': {0: {'label': 5, 'accuracy': 0.86352986, 'prediction': [0.367604, 0.31575006, 0.4197184, 0.42311758]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/cleveland_1534910804675399205_20170611_jpg.rf.0a3607e06cb

{'miami_1087121647915855414_20151002_jpg.rf.af494e8f92838697bc7fbd113cc58da9.jpg': {0: {'label': 5, 'accuracy': 0.7603428, 'prediction': [0.8757277, 0.20678028, 0.9456945, 0.36561278]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/la_2017_1675915703935724271_20171223_jpg.rf.1664f4cc0e484e53f4cb61d66f7be754.jpg
{'la_2017_1675915703935724271_20171223_jpg.rf.1664f4cc0e484e53f4cb61d66f7be754.jpg': {0: {'label': 2, 'accuracy': 0.8986497, 'prediction': [0.45297167, 0.4737647, 0.5406726, 0.56832767]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR/miami_1924312589946641684_20181130_jpg.rf.bbfd1110f958ef22fe52646b3640cf18.jpg
{'miami_1924312589946641684_20181130_jpg.rf.bbfd1110f958ef22fe52646b3640cf18.jpg': {0: {'label': 1, 'accuracy': 0.68912643, 'prediction': [0.27814397, 0.5509889, 0.3362064, 0.6263912]}}}
{'miami_1924312589946641684_20181130_jpg.rf.bbfd1110f958ef22fe52646b3640cf18.jpg': {1: {'label': 1, 'accuracy': 0.6803796, 'predictio